In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import re
import time

In [2]:
# 크롬옵션
chrome_options = Options()
# 꺼짐 방지
chrome_options.add_argument('--disable-gpu') 
# 알림표시 제거
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)

url = '''https://sminfo.mss.go.kr/cm/sv/CSV001R0.do'''
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

time.sleep(0.1)
first_window_handle = driver.window_handles[0]

driver.switch_to.window(first_window_handle)
time.sleep(0.1)
id_input = driver.find_element(By.ID, "login_id")
id_input.send_keys("anabiotic")

password_input = driver.find_element(By.ID, "login_password")
password_input.send_keys("qwe920435!")

password_input.send_keys(Keys.ENTER)

In [3]:
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/a').click()
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/ul/li[1]/a').click()

In [4]:
search_input = driver.find_element_by_id("searchTxt")
search_input.send_keys("검색어 입력")

AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

In [4]:
driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/form[4]/div/div[4]/button[1]').click()

In [5]:
driver.find_element(By.XPATH, '/html/body').send_keys(Keys.PAGE_DOWN)

In [28]:
# 설립연도
int(''.join(re.findall('\\d\\d\\d\\d' ,driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[2]/td[2]').text)))

2013

In [23]:
# 주소
' '.join(re.findall(r'\b(\w+[구시도남군])\b', driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[4]/td').text))

'전남 무안군'

In [9]:
# 매출(단위 = 천원) 을 만원단위로
int(int(driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/div[5]/table/tbody/tr[1]/td[5]').text.replace(',',''))/10)

652178

In [38]:
# 기업 URL
driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[5]/td[1]/a').text

'www.nbt.com'

In [39]:
# 표준산업
driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[6]/td[1]').text

'광고매체 판매업'

In [43]:
data = pd.read_csv('완벽.csv', index_col=0)
corp_list = data.기업명

In [48]:
corp_list = corp_list.drop_duplicates()

In [41]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import time

# 크롬옵션
chrome_options = Options()
# 꺼짐 방지
chrome_options.add_argument('--disable-gpu') 
# 알림표시 제거
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)

# 데이터 준비
data = pd.read_csv('완벽.csv', index_col=0)

corp_list = data.기업명
test_list = []
try:    
    for zxc in data.주소:
        test_list.append(zxc[5:8].strip())
except:
    test_list.append(zxc)
data_1 = pd.DataFrame({
    '기업명' : corp_list,
    '주소' : test_list
    })
data_1 = data_1.drop_duplicates(subset=['기업명'])

sm_res = pd.DataFrame(columns=['기업명', '설립연도', '주소', '매출액', '기업 홈페이지 URL', '업종분류'])

# URL 입력
url = '''https://sminfo.mss.go.kr/cm/sv/CSV001R0.do'''
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 팝업 무시
time.sleep(.1)
first_window_handle = driver.window_handles[0]

# 로그인
driver.switch_to.window(first_window_handle)
time.sleep(.1)
id_input = driver.find_element(By.ID, "login_id")
id_input.send_keys("anabiotic")
time.sleep(.1)
password_input = driver.find_element(By.ID, "login_password")
password_input.send_keys("qwe920435!")
time.sleep(.1)
password_input.send_keys(Keys.ENTER)
time.sleep(.3)
# 기업정보보기 클릭
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/a').click()
time.sleep(.1)
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/ul/li[1]/a').click()
# 기업 리스트 검색
try:
    time.sleep(2)
    for corp_name, address_name in zip(data_1.기업명, data_1.주소):
        driver.find_element(By.ID, "searchTxt").send_keys(f"{corp_name}")
        driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/form[4]/div/div[4]/button[1]').click()

        # 클릭 위해서 페이지 다운
        time.sleep(.1) 
        driver.find_element(By.XPATH, '/html/body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        # 검색어 검증
        try:
            for cat_list in range(1,11):
                if corp_name in driver.find_element(By.XPATH, f'//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr[{cat_list}]/td[1]/a').text and address_name in driver.find_element(By.XPATH, f'//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr[{cat_list}]/td[5]').text:
                    driver.find_element(By.XPATH, f'//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr[{cat_list}]/td[1]/a').click()
                    break
                else:
                    pass
        except:
            pass

        # data_crawling
        time.sleep(3)

        # 설립연도
        try:
            corp_year = int(''.join(re.findall('\\d\\d\\d\\d' ,driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[2]/td[2]').text)))
        except:
            corp_year = None
        # 주소
        try:
            address = ' '.join(re.findall(r'\b(\w+[구시도남군])\b', driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[4]/td').text))
        except:
            address = None
        # 매출(단위 = 천원)
        try:
            sales = int(int(driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/div[5]/table/tbody/tr[1]/td[5]').text.replace(',',''))/10)
        except:
            sales = None        
        # 기업 URL
        try:
            corp_URL = driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[5]/td[1]/a').text
        except:
            corp_URL = None        
        # 표준산업
        try:    
            job_dis = driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[6]/td[1]').text
        except:
            job_dis = None        
        
        sm_temp = pd.DataFrame([corp_name, corp_year, address, sales, corp_URL, job_dis], index = ['기업명', '설립연도', '주소', '매출액', '기업 홈페이지 URL', '업종분류'])
        sm_res = pd.concat([sm_res, sm_temp])
        
        time.sleep(3)
        
        driver.back()

except Exception as ex:
    print(corp_name, ex)
    pass


세이프에이아이 Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.70)
Stacktrace:
	GetHandleVerifier [0x00007FF6D5EDAD22+56930]
	(No symbol) [0x00007FF6D5E4F622]
	(No symbol) [0x00007FF6D5D042E5]
	(No symbol) [0x00007FF6D5CE1D4C]
	(No symbol) [0x00007FF6D5D723F7]
	(No symbol) [0x00007FF6D5D87891]
	(No symbol) [0x00007FF6D5D6BA43]
	(No symbol) [0x00007FF6D5D3D438]
	(No symbol) [0x00007FF6D5D3E4D1]
	GetHandleVerifier [0x00007FF6D6256AAD+3709933]
	GetHandleVerifier [0x00007FF6D62AFFED+4075821]
	GetHandleVerifier [0x00007FF6D62A817F+4043455]
	GetHandleVerifier [0x00007FF6D5F79756+706710]
	(No symbol) [0x00007FF6D5E5B8FF]
	(No symbol) [0x00007FF6D5E56AE4]
	(No symbol) [0x00007FF6D5E56C3C]
	(No symbol) [0x00007FF6D5E468F4]
	BaseThreadInitThunk [0x00007FFCB755257D+29]
	RtlUserThreadStart [0x00007FFCB7B6AA58+40]



In [32]:
def process_corporate_data_twice(corp_name, address_name):
    sm_res = pd.DataFrame(columns=['기업명', '설립연도', '주소', '매출액', '기업 홈페이지 URL', '업종분류'])
    try:
        time.sleep(2)
        driver.find_element(By.ID, "searchTxt").send_keys(f"{corp_name}")
        driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/form[4]/div/div[4]/button[1]').click()
        # 클릭 위해서 페이지 다운
        time.sleep(0.1) 
        driver.find_element(By.XPATH, '/html/body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        # 검색어 검증
        try:
            for cat_list in range(1, 11):
                if corp_name in driver.find_element(By.XPATH, f'//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr[{cat_list}]/td[1]/a').text and address_name in driver.find_element(By.XPATH, f'//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr[{cat_list}]/td[5]').text:
                    driver.find_element(By.XPATH, f'//*[@id="contents_sub"]/div[3]/form[4]/div/div[7]/table/tbody/tr[{cat_list}]/td[1]/a').click()
                    break
        except:
            driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/a').click()
            time.sleep(0.1)
            driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/ul/li[1]/a').click()
            pass
        # data_crawling
        time.sleep(3)
        # 설립연도
        try:
            corp_year = int(''.join(re.findall('\\d\\d\\d\\d' ,driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[2]/td[2]').text)))
        except:
            corp_year = None
        # 주소
        try:
            address = ' '.join(re.findall(r'\b(\w+[구시도남군])\b', driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[4]/td').text))
        except:
            address = None
        # 매출(단위 = 천원)
        try:
            sales = int(int(driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/div[5]/table/tbody/tr[1]/td[5]').text.replace(',',''))/10)
        except:
            sales = None        
        # 기업 URL
        try:
            corp_URL = driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[5]/td[1]/a').text
        except:
            corp_URL = None        
        # 표준산업
        try:    
            job_dis = driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[6]/td[1]').text
        except:
            job_dis = None        
        sm_temp = pd.DataFrame([[corp_name, corp_year, address, sales, corp_URL, job_dis]], columns=['기업명', '설립연도', '주소', '매출액', '기업 홈페이지 URL', '업종분류'])
        sm_res = pd.concat([sm_res, sm_temp])
        
        time.sleep(.2)
        driver.back()

        time.sleep(28)  # 35초 대기
        return sm_res
    
    except Exception as ex:
        print(corp_name, ex)
        pass
        time.sleep(35)  # 35초 대기
        return sm_res
        


In [33]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import re
import time

# 데이터 준비
data = pd.read_csv('완벽.csv', index_col=0)

corp_list = data.기업명
test_list = []
try:    
    for zxc in data.주소:
        test_list.append(zxc[5:8].strip())
except:
    test_list.append(zxc)
data_1 = pd.DataFrame({
    '기업명' : corp_list,
    '주소' : test_list
})
data_1 = data_1.drop_duplicates(subset=['기업명'])
data_1.기업명 = data_1.기업명.apply(lambda x: re.sub('\\W', '', x))

# 크롬옵션
chrome_options = Options()
chrome_options.add_argument('--disable-gpu') 
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=chrome_options)
    
# URL 입력
url = '''https://sminfo.mss.go.kr/cm/sv/CSV001R0.do'''
driver.get(url)
# 팝업 무시
time.sleep(0.1)
first_window_handle = driver.window_handles[0]
# 로그인
driver.switch_to.window(first_window_handle)
time.sleep(0.1)
id_input = driver.find_element(By.ID, "login_id")
id_input.send_keys("anabiotic")
time.sleep(0.1)
password_input = driver.find_element(By.ID, "login_password")
password_input.send_keys("qwe920435!")
time.sleep(0.1)
password_input.send_keys(Keys.ENTER)
time.sleep(0.3)
# 기업정보보기 클릭
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/a').click()
time.sleep(0.1)
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul/li[1]/ul/li[1]/a').click()
    
sm_res = pd.DataFrame(columns=['기업명', '설립연도', '주소', '매출액', '기업 홈페이지 URL', '업종분류'])

# 기업 리스트 처리
for corp_name, address_name in zip(data_1.기업명, data_1.주소):
    res = process_corporate_data_twice(corp_name, address_name)
    sm_res = pd.concat([sm_res, res])

In [35]:
sm_res.to_excel('잡것완성.xlsx')

In [36]:
sm_res.isna().sum()

기업명              0
설립연도           214
주소             208
매출액            246
기업 홈페이지 URL    350
업종분류           208
dtype: int64

In [3]:
dd = pd.read_excel('잡것완성.xlsx', index_col = 0)

In [6]:
ddd = pd.read_csv('wanted_data.csv', index_col= 0 )

In [7]:
ddd.기업명 = ddd.기업명.apply(lambda x: re.sub(r'\([^)]*\)','',x))
ddd.기업명 = ddd.기업명.apply(lambda x: re.sub('\\W', '', x))

In [11]:

ddd[ddd.기업명 == '플라네타리움랩스코리아']

,기업명,주소,직무,자격요건,직급,이용하는기술스택/우대사항,해당 페이지 URL
213,플라네타리움랩스코리아,강남구 테헤란로 223,소프트웨어 엔지니어,원자격•학력및성별무관•관련경력2년이상및그에준하는경력요구사항•1개이상의프로그래밍언어로...,2,"• 오픈 소스 프로젝트 메인테이너, 혹은 기여 경험, • 2인 이상 팀에서 협업 해...",https://www.wanted.co.kr/wd/199628
497,플라네타리움랩스코리아,강남구 테헤란로 223,웹 개발자,원자격•학력및성별무관•관련경력2년이상및그에준하는경력요구사항•1개이상의프로그래밍언어로...,2,"• 오픈 소스 프로젝트 메인테이너, 혹은 기여 경험, • 2인 이상 팀에서 협업 해...",https://www.wanted.co.kr/wd/199628
1823,플라네타리움랩스코리아,강남구 테헤란로 223,블록체인 플랫폼 엔지니어,원자격•학력및성별무관•관련경력2년이상및그에준하는경력요구사항•1개이상의프로그래밍언어로...,2,"• 오픈 소스 프로젝트 메인테이너, 혹은 기여 경험, • 2인 이상 팀에서 협업 해...",https://www.wanted.co.kr/wd/199628
